<a href="https://colab.research.google.com/github/inubasfloran/machine-learning/blob/master/MachineLearning_LTSM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install KiteConnect
!pip install pandas_ta
from kiteconnect import KiteConnect
from datetime import datetime
import pandas as pd

import pandas_ta as ta
kite = KiteConnect(api_key="########")
kite.set_access_token("##########")
#https://ieeexplore.ieee.org/abstract/document/9165760/metrics#metrics
from_date = datetime.strptime('2022-05-12 9:15:00', '%Y-%m-%d %H:%M:%S')
to_date = datetime.strptime('2022-05-12 10:15:00', '%Y-%m-%d %H:%M:%S')
historical_data = kite.historical_data(instrument_token=str(6401),from_date=from_date,to_date=to_date,interval='minute', oi=True)


In [48]:

data = pd.DataFrame(historical_data)
data.set_index(pd.DatetimeIndex(data["date"]),inplace=True)
data.ta.sma(9,append=True)
data.ta.wma(9,append=True)
data.ta.stoch(k=14,smooth_k=1,smooth_d=3,append=True)
data=pd.concat([ta.mom(data['open'],length=10),data],axis=1)
data.ta.rsi(14,append=True)
data.ta.macd(fast=12,slow=26,signal=9,append=True)
data.ta.willr(length=14,append=True)
#Need to check about this indicator
data.ta.adosc(append=True)
data.ta.cci(20,append=True)
print(data.tail(30))


                           MOM_10                      date     open     high  \
date                                                                            
2022-05-12 09:46:00+05:30    1.35 2022-05-12 09:46:00+05:30  2092.05  2092.90   
2022-05-12 09:47:00+05:30    3.15 2022-05-12 09:47:00+05:30  2090.40  2094.00   
2022-05-12 09:48:00+05:30   11.55 2022-05-12 09:48:00+05:30  2094.00  2097.50   
2022-05-12 09:49:00+05:30   10.90 2022-05-12 09:49:00+05:30  2097.45  2097.45   
2022-05-12 09:50:00+05:30   10.00 2022-05-12 09:50:00+05:30  2095.00  2097.00   
2022-05-12 09:51:00+05:30    7.00 2022-05-12 09:51:00+05:30  2093.00  2097.00   
2022-05-12 09:52:00+05:30    6.90 2022-05-12 09:52:00+05:30  2092.65  2095.00   
2022-05-12 09:53:00+05:30    2.65 2022-05-12 09:53:00+05:30  2092.75  2097.00   
2022-05-12 09:54:00+05:30    8.20 2022-05-12 09:54:00+05:30  2095.75  2098.00   
2022-05-12 09:55:00+05:30    3.20 2022-05-12 09:55:00+05:30  2093.55  2094.00   
2022-05-12 09:56:00+05:30   

In [49]:
#Step2 - Change indicators to 0 and 1
data = data.dropna()
import numpy as np

data['BI_SMA_9'] = np.where(data['close'] < data['SMA_9'] , -1 , 1 )
data['BI_WMA_9'] = np.where(data['close'] < data['WMA_9'] , -1 , 1 )
data['BI_MOM_10'] = np.where(data['MOM_10'] < 0 , -1 , 1 )
data['BI_STOCHk_14_3_1'] = np.where(data['STOCHk_14_3_1' ] < data['STOCHk_14_3_1'].shift() , -1 , 1)
data['BI_STOCHd_14_3_1'] = np.where(data['STOCHd_14_3_1' ] < data['STOCHd_14_3_1'].shift() , -1 , 1)
data['BI_WILLR_14'] = np.where(data['WILLR_14' ] < data['WILLR_14'].shift() , -1 , 1)
data['BI_MACD_12_26_9'] = np.where(data['MACD_12_26_9' ] < data['MACD_12_26_9'].shift() , -1 , 1)
data['BI_ADOSC_3_10'] = np.where(data['ADOSC_3_10' ] < data['ADOSC_3_10'].shift() , -1 , 1)
data['BI_RSI_14'] = np.select([(data['RSI_14' ] >= 70) ,(data['RSI_14'] <= 30) , (data['RSI_14']  > 30) & (data['RSI_14'] < 70) 
& (data['RSI_14'] < data['RSI_14'].shift()) , (data['RSI_14']  > 30) & (data['RSI_14'] < 70) & (data['RSI_14'] > data['RSI_14'].shift()) ] , [-1 , 1 , -1 , 1]  )
data['BI_CCI_20_0.015'] = np.select([(data['CCI_20_0.015'] > 200) ,(data['CCI_20_0.015'] < -200) 
, ((data['CCI_20_0.015'] < 200) & (data['CCI_20_0.015'] > -200) & (data['CCI_20_0.015'] < data['CCI_20_0.015'].shift())  )
, ((data['CCI_20_0.015'] < 200) & (data['CCI_20_0.015'] > -200) & (data['CCI_20_0.015'] > data['CCI_20_0.015'].shift())  ) ] , [-1 , +1 , -1 , +1])

data[['close','BI_SMA_9','BI_WMA_9','BI_MOM_10','BI_STOCHk_14_3_1','BI_STOCHd_14_3_1','BI_WILLR_14','BI_ADOSC_3_10','BI_RSI_14','BI_CCI_20_0.015']]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

,close,BI_SMA_9,BI_WMA_9,BI_MOM_10,BI_STOCHk_14_3_1,BI_STOCHd_14_3_1,BI_WILLR_14,BI_ADOSC_3_10,BI_RSI_14,BI_CCI_20_0.015
date,,,,,,,,,,
2022-05-12 09:48:00+05:30,2097.45,1,1,1,1,1,1,1,0,0
2022-05-12 09:49:00+05:30,2095.00,1,1,1,-1,1,-1,1,-1,-1
2022-05-12 09:50:00+05:30,2093.95,1,1,1,-1,-1,-1,-1,-1,-1
2022-05-12 09:51:00+05:30,2093.30,1,-1,1,-1,-1,-1,-1,-1,-1
2022-05-12 09:52:00+05:30,2094.35,1,1,1,1,-1,1,-1,1,-1
2022-05-12 09:53:00+05:30,2095.75,1,1,1,1,1,1,1,1,1
2022-05-12 09:54:00+05:30,2093.55,-1,-1,1,-1,-1,-1,-1,-1,-1
2022-05-12 09:55:00+05:30,2090.30,-1,-1,1,-1,-1,-1,-1,-1,-1
2022-05-12 09:56:00+05:30,2087.70,-1,-1,-1,-1,-1,-1,-1,-1,-1
